<a href="https://colab.research.google.com/github/ParkWonjeong/Limitless/blob/main/Project_Limitless_Trading_bot_log.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
# 금융 데이터 수집 및 차트 작성을 위한 라이브러리 설치
!pip install yfinance pandas_ta plotly

import yfinance as yf
import pandas as pd
import pandas_ta as ta
import plotly.graph_objects as go
import plotly.io as pio
# Colab 전용 렌더러로 설정 (차트가 안 보일 때 해결법)
pio.renderers.default = 'colab'

In [27]:
# 비트코인(BTC-USD) 데이터 가져오기 (1시간 봉 기준)
df = yf.download("BTC-USD", period="1mo", interval="1h")

# 데이터 확인
print(df.tail())

/tmp/ipython-input-3682059198.py:2: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed

Price                             Close          High           Low  \
Ticker                          BTC-USD       BTC-USD       BTC-USD   
Datetime                                                              
2026-01-30 10:00:00+00:00  82657.203125  82722.351562  82057.585938   
2026-01-30 11:00:00+00:00  83055.718750  83098.054688  82565.000000   
2026-01-30 12:00:00+00:00  82632.265625  83167.117188  82546.523438   
2026-01-30 13:00:00+00:00  82570.156250  82815.703125  82459.289062   
2026-01-30 14:00:00+00:00  82826.546875  82923.585938  82610.843750   

Price                              Open      Volume  
Ticker                          BTC-USD     BTC-USD  
Datetime                                             
2026-01-30 10:00:00+00:00  82225.156250  1975410688  
2026-01-30 11:00:00+00:00  82689.851562  1156489216  
2026-01-30 12:00:00+00:00  83043.054688  6237896704  
2026-01-30 13:00:00+00:00  82653.062500  7121764352  
2026-01-30 14:00:00+00:00  82610.843750  1680637952  

In [28]:
# Colab 출력 강제 설정
pio.renderers.default = 'colab'

# 데이터 다시 불러오기 및 정리
df = yf.download("BTC-USD", period="7d", interval="1h")
df.columns = df.columns.droplevel('Ticker') # 다중 인덱스 해결

# 차트 생성
fig = go.Figure(data=[go.Candlestick(
    x=df.index,
    open=df['Open'],
    high=df['High'],
    low=df['Low'],
    close=df['Close']
)])

fig.update_layout(
    title='비트코인(BTC-USD) 1시간 봉 차트',
    template='plotly_dark',
    xaxis_rangeslider_visible=False
)

fig.show()

/tmp/ipython-input-4036211935.py:5: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed


## 이동평균선 추가

In [29]:
import pandas_ta as ta # 지표 계산을 위한 라이브러리

# 1. 이동평균선 계산
# df['Close'] 데이터가 한 줄(Series)인지 확인 후 계산
df['MA20'] = ta.sma(df['Close'], length=20)
df['MA50'] = ta.sma(df['Close'], length=50)

# 2. 기존 캔들스틱 차트에 선 추가하기
fig = go.Figure(data=[go.Candlestick(
    x=df.index,
    open=df['Open'], high=df['High'],
    low=df['Low'], close=df['Close'],
    name="Candlestick"
)])

# MA20 선 추가 (주황색)
fig.add_trace(go.Scatter(x=df.index, y=df['MA20'],
                         mode='lines', name='MA 20',
                         line=dict(color='orange', width=1.5)))

# MA50 선 추가 (하늘색)
fig.add_trace(go.Scatter(x=df.index, y=df['MA50'],
                         mode='lines', name='MA 50',
                         line=dict(color='cyan', width=1.5)))

# 3. 레이아웃 설정
fig.update_layout(
    title='BTC-USD 이동평균선 전략 차트',
    template='plotly_dark',
    xaxis_rangeslider_visible=False
)

fig.show()

## RSI 지표 추가

In [30]:
from plotly.subplots import make_subplots

# 1. Colab 출력 및 데이터 준비
pio.renderers.default = 'colab'
df = yf.download("BTC-USD", period="7d", interval="1h")
df.columns = df.columns.droplevel('Ticker') # 다중 인덱스 제거

# 2. 지표 계산 (MA + RSI)
df['MA20'] = ta.sma(df['Close'], length=20)
df['MA50'] = ta.sma(df['Close'], length=50)
df['RSI'] = ta.rsi(df['Close'], length=14) # 기본값인 14일 기준

# 3. 차트 레이아웃 설정 (2층 차트 만들기)
fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                    vertical_spacing=0.1,
                    subplot_titles=('BTC 가격 및 이동평균선', 'RSI 지표'),
                    row_heights=[0.7, 0.3]) # 7:3 비율

# 4. 상단(1행): 캔들스틱 및 이동평균선 추가
fig.add_trace(go.Candlestick(
    x=df.index, open=df['Open'], high=df['High'],
    low=df['Low'], close=df['Close'], name="BTC"), row=1, col=1)

fig.add_trace(go.Scatter(x=df.index, y=df['MA20'], name='MA20', line=dict(color='orange')), row=1, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df['MA50'], name='MA50', line=dict(color='cyan')), row=1, col=1)

# 5. 하단(2행): RSI 추가
fig.add_trace(go.Scatter(x=df.index, y=df['RSI'], name='RSI', line=dict(color='purple')), row=2, col=1)

# RSI 기준선(30, 70) 추가
fig.add_hline(y=70, line_dash="dash", line_color="red", row=2, col=1)
fig.add_hline(y=30, line_dash="dash", line_color="green", row=2, col=1)

# 6. 마무리 설정
fig.update_layout(
    height=800,
    title='Clowder 봇: 기술적 분석 대시보드',
    template='plotly_dark',
    xaxis_rangeslider_visible=False
)

fig.show()

/tmp/ipython-input-3773776558.py:5: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed


In [31]:
import yfinance as yf
import pandas as pd
import pandas_ta as ta
import plotly.graph_objects as go
import plotly.io as pio

# 1. 환경 설정 및 데이터 수집
pio.renderers.default = 'colab'
df = yf.download("BTC-USD", period="3mo", interval="1d") # 스윙 전략이므로 일봉(1d) 기준
df.columns = df.columns.droplevel('Ticker')

# 2. Smart Swing v4 기술적 지표 계산
df['MA5'] = ta.sma(df['Close'], length=5)
df['MA20'] = ta.sma(df['Close'], length=20)

# 3. 매수 신호(Golden Cross) 로직
# 5일선이 20일선을 상향 돌파할 때
df['Signal'] = (df['MA5'] > df['MA20']) & (df['MA5'].shift(1) <= df['MA20'].shift(1))

# 4. 차트 시각화
fig = go.Figure()

# 캔들스틱 추가
fig.add_trace(go.Candlestick(
    x=df.index, open=df['Open'], high=df['High'],
    low=df['Low'], close=df['Close'], name="BTC Price"))

# 이동평균선 추가 (5일, 20일)
fig.add_trace(go.Scatter(x=df.index, y=df['MA5'], name='MA5 (단기)', line=dict(color='yellow', width=1.5)))
fig.add_trace(go.Scatter(x=df.index, y=df['MA20'], name='MA20 (장기)', line=dict(color='cyan', width=1.5)))

# 매수 신호 지점 표시
buy_signals = df[df['Signal'] == True]
fig.add_trace(go.Scatter(
    x=buy_signals.index, y=buy_signals['Low'] * 0.98,
    mode='markers', name='BUY Signal',
    marker=dict(symbol='triangle-up', size=12, color='lime')
))

# 5. [핵심] 리스크 관리 라인 시각화 (가장 최근 신호 기준)
if not buy_signals.empty:
    latest_buy_price = buy_signals['Close'].iloc[-1]
    tp_price = latest_buy_price * 1.15 # 익절가 +15%
    sl_price = latest_buy_price * 0.98 # 손절가 -2%

    # 익절선 표시
    fig.add_hline(y=tp_price, line_dash="dot", line_color="green",
                  annotation_text=f"Target (+15%): {tp_price:.0f}")
    # 손절선 표시
    fig.add_hline(y=sl_price, line_dash="dot", line_color="red",
                  annotation_text=f"Stop Loss (-2%): {sl_price:.0f}")

# 레이아웃 설정
fig.update_layout(
    title='Smart Swing v4 전략 대시보드 (AI + Technical Analysis)',
    template='plotly_dark',
    xaxis_rangeslider_visible=False,
    height=700
)

fig.show()

/tmp/ipython-input-395617886.py:9: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed


현재 기술적 지표만 추가한 상태에서의 백테스팅

In [32]:
import yfinance as yf
import pandas as pd
import pandas_ta as ta

# 1. 데이터 준비 (최근 1년 일봉 데이터)
df = yf.download("BTC-USD", period="1y", interval="1d")
df.columns = df.columns.droplevel('Ticker')

# 2. 지표 계산
df['MA5'] = ta.sma(df['Close'], length=5)
df['MA20'] = ta.sma(df['Close'], length=20)

# 3. 백테스팅 변수 설정
initial_balance = 10000000  # 초기 자본 1,000만원
balance = initial_balance
position = 0                # 보유 수량
buy_price = 0               # 매수 가격
history = []                # 수익률 기록

# 4. 시뮬레이션 루프
for i in range(1, len(df)):
    current_price = df['Close'].iloc[i]
    prev_ma5 = df['MA5'].iloc[i-1]
    prev_ma20 = df['MA20'].iloc[i-1]
    curr_ma5 = df['MA5'].iloc[i]
    curr_ma20 = df['MA20'].iloc[i]

    # [매수 조건]: 골든크로스 발생 시 & 현재 무포지션일 때
    if position == 0 and curr_ma5 > curr_ma20 and prev_ma5 <= prev_ma20:
        buy_price = current_price
        position = balance / buy_price
        balance = 0
        print(f"[{df.index[i].date()}] 매수 | 가격: {buy_price:,.0f}")

    # [매도 조건]: 포지션 보유 중일 때
    elif position > 0:
        profit_loss = (current_price - buy_price) / buy_price

        # 1) 익절: +15% 이상 달성
        # 2) 손절: -2% 이하 하락
        # 3) 추세 이탈: 데드크로스 발생 (추가 필터)
        if profit_loss >= 0.15 or profit_loss <= -0.02 or (curr_ma5 < curr_ma20):
            balance = position * current_price
            history.append(profit_loss)
            status = "익절" if profit_loss >= 0.15 else "손절" if profit_loss <= -0.02 else "추세이탈"
            print(f"[{df.index[i].date()}] 매도 ({status}) | 가격: {current_price:,.0f} | 수익률: {profit_loss*100:.2f}%")
            position = 0
            buy_price = 0

# 최종 정산
final_assets = balance if position == 0 else position * df['Close'].iloc[-1]
total_return = (final_assets - initial_balance) / initial_balance * 100
win_rate = len([x for x in history if x > 0]) / len(history) * 100 if history else 0

print("\n" + "="*30)
print(f"▶ 최종 자산: {final_assets:,.0f}원")
print(f"▶ 누적 수익률: {total_return:.2f}%")
print(f"▶ 매매 횟수: {len(history)}회")
print(f"▶ 승률: {win_rate:.2f}%")
print("="*30)

/tmp/ipython-input-4019243290.py:6: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed

[2025-02-23] 매수 | 가격: 96,274
[2025-02-24] 매도 (손절) | 가격: 91,418 | 수익률: -5.04%
[2025-03-23] 매수 | 가격: 86,054
[2025-03-29] 매도 (손절) | 가격: 82,598 | 수익률: -4.02%
[2025-04-14] 매수 | 가격: 84,542
[2025-05-08] 매도 (익절) | 가격: 103,241 | 수익률: 22.12%
[2025-06-10] 매수 | 가격: 110,257
[2025-06-12] 매도 (손절) | 가격: 105,929 | 수익률: -3.93%
[2025-06-27] 매수 | 가격: 107,088
[2025-07-29] 매도 (추세이탈) | 가격: 117,922 | 수익률: 10.12%
[2025-08-10] 매수 | 가격: 119,307
[2025-08-18] 매도 (손절) | 가격: 116,252 | 수익률: -2.56%
[2025-09-10] 매수 | 가격: 113,955
[2025-09-24] 매도 (추세이탈) | 가격: 113,329 | 수익률: -0.55%
[2025-10-02] 매수 | 가격: 120,681
[2025-10-10] 매도 (손절) | 가격: 113,214 | 수익률: -6.19%
[2025-10-27] 매수 | 가격: 114,119
[2025-10-29] 매도 (손절) | 가격: 110,055 | 수익률: -3.56%
[2025-12-03] 매수 | 가격: 93,528
[2025-12-05] 매도 (손절) | 가격: 89,388 | 수익률: -4.43%
[2026-01-01] 매수 | 가격: 88,732
[2026-01-21] 매도 (추세이탈) | 가격: 89,377 | 수익률: 0.73%

▶ 최종 자산: 9,937,940원
▶ 누적 수익률: -0.62%
▶ 매매 횟수: 11회
▶ 승률: 27.27%


## 딥러닝 모델 추가

In [33]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# 1. 데이터 수집 (최근 5년)
data = yf.download("BTC-USD", period="5y", interval="1d")
data.columns = data.columns.droplevel('Ticker')

# 2. 데이터 정규화 (0~1 사이 값으로 변환)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data['Close'].values.reshape(-1,1))

# 3. 학습 데이터 생성 (과거 60일의 데이터를 보고 다음 날을 예측)
prediction_days = 60
x_train, y_train = [], []

for x in range(prediction_days, len(scaled_data)):
    x_train.append(scaled_data[x-prediction_days:x, 0])
    y_train.append(scaled_data[x, 0])

x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

/tmp/ipython-input-2643635898.py:9: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed


In [34]:
model = Sequential()

# 첫 번째 LSTM 레이어
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(Dropout(0.2)) # 과적합 방지

# 두 번째 LSTM 레이어
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))

# 출력 레이어 (내일 가격 예측)
model.add(Dense(units=1))

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, epochs=25, batch_size=32)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Epoch 1/25
56/56 ━━━━━━━━━━━━━━━━━━━━ 13s 92ms/step - loss: 0.0773
Epoch 2/25
56/56 ━━━━━━━━━━━━━━━━━━━━ 3s 49ms/step - loss: 0.0031
Epoch 3/25
56/56 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - loss: 0.0030
Epoch 4/25
56/56 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - loss: 0.0027
Epoch 5/25
56/56 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - loss: 0.0028
Epoch 6/25
56/56 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - loss: 0.0025
Epoch 7/25
56/56 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step - loss: 0.0026
Epoch 8/25
56/56 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - loss: 0.0026
Epoch 9/25
56/56 ━━━━━━━━━━━━━━━━━━━━ 3s 36ms/step - loss: 0.0023
Epoch 10/25
56/56 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - loss: 0.0022
Epoch 11/25
56/56 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - loss: 0.0021
Epoch 12/25
56/56 ━━━━━━━━━━━━━━━━━━━━ 3s 49ms/step - loss: 0.0022
Epoch 13/25
56/56 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - loss: 0.0022
Epoch 14/25
56/56 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - loss: 0.0022
Epoch 15/25
56/56 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - loss: 0.0019
Epo

In [35]:
import numpy as np

# 1. 초기 설정
balance = 10000000
position = 0
history = []

# 2. 백테스팅 루프 (학습 데이터 이후 시점부터 시작)
for i in range(prediction_days, len(df)):
    current_price = df['Close'].iloc[i]

    # [AI 예측 구간]
    # 현재 시점까지의 데이터를 스케일러로 변환 (이미 학습 때 만든 scaled_data 활용)
    last_60_days = scaled_data[i-60:i, 0]
    last_60_days_input = np.reshape(last_60_days, (1, 60, 1))
    pred_scaled = model.predict(last_60_days_input, verbose=0)
    pred_price = scaler.inverse_transform(pred_scaled)[0][0]

    # 지표 데이터
    curr_ma5 = df['MA5'].iloc[i]
    curr_ma20 = df['MA20'].iloc[i]
    prev_ma5 = df['MA5'].iloc[i-1]
    prev_ma20 = df['MA20'].iloc[i-1]

    # [매수 조건: 기술적 지표 + AI 필터]
    # AI 조건: 내일 예측가가 오늘 종가보다 높을 것 (상승 예측)
    is_ai_bullish = pred_price > current_price
    is_golden_cross = (curr_ma5 > curr_ma20 and prev_ma5 <= prev_ma20)

    if is_golden_cross:
      print(f"[{df.index[i].date()}] 골든크로스 발생! 하지만 AI의 판단은? -> {'상승예측' if is_ai_bullish else '하락예측'}")

    if position == 0 and is_golden_cross and is_ai_bullish:
        buy_price = current_price
        position = balance / buy_price
        balance = 0
        print(f"[{df.index[i].date()}] AI승인 매수 | 가격: {buy_price:,.0f} (예측가: {pred_price:,.0f})")

    # [매도 조건: 기존과 동일]
    elif position > 0:
        profit_loss = (current_price - buy_price) / buy_price
        if profit_loss >= 0.15 or profit_loss <= -0.02 or (curr_ma5 < curr_ma20):
            balance = position * current_price
            history.append(profit_loss)
            position = 0
            print(f"[{df.index[i].date()}] 매도 완료 | 수익률: {profit_loss*100:.2f}%")

# 최종 결과 출력
print(f"\n최종 수익률: {(balance - 10000000)/100000:.2f}%")

[2025-04-14] 골든크로스 발생! 하지만 AI의 판단은? -> 하락예측
[2025-06-10] 골든크로스 발생! 하지만 AI의 판단은? -> 하락예측
[2025-06-27] 골든크로스 발생! 하지만 AI의 판단은? -> 하락예측
[2025-08-10] 골든크로스 발생! 하지만 AI의 판단은? -> 하락예측
[2025-09-10] 골든크로스 발생! 하지만 AI의 판단은? -> 하락예측
[2025-10-02] 골든크로스 발생! 하지만 AI의 판단은? -> 하락예측
[2025-10-27] 골든크로스 발생! 하지만 AI의 판단은? -> 하락예측
[2025-12-03] 골든크로스 발생! 하지만 AI의 판단은? -> 하락예측
[2026-01-01] 골든크로스 발생! 하지만 AI의 판단은? -> 하락예측

최종 수익률: 0.00%


모델 변경: LSTM -> GRU

In [36]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout

# [이 부분이 핵심!] 모델의 구조를 GRU로 정의합니다.
model = Sequential()

# 첫 번째 GRU 레이어 (입력 데이터의 피처가 3개라고 가정: Price, RSI, Volume)
model.add(GRU(units=50, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(Dropout(0.2))

# 두 번째 GRU 레이어
model.add(GRU(units=50, return_sequences=False))
model.add(Dropout(0.2))

# 출력 레이어 (0~1 사이의 확률을 뱉도록 Sigmoid 사용)
model.add(Dense(units=1, activation='sigmoid'))

# 컴파일 (분류 문제이므로 binary_crossentropy 권장)
model.compile(optimizer='adam', loss='binary_crossentropy')

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



In [37]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# 1. Early Stopping 설정
# monitor='val_loss': 검증 오차를 관찰
# patience=10: 오차가 개선되지 않더라도 10번은 더 지켜봄
# restore_best_weights=True: 학습 중단 후 가장 성적이 좋았던 가중치로 복구
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

# 2. 가장 좋은 모델을 파일로 저장하는 설정 (선택 사항)
checkpoint = ModelCheckpoint(
    'best_limitless_model.h5',
    monitor='val_loss',
    save_best_only=True
)

# 3. 모델 학습 실행
history = model.fit(
    x_train, y_train,
    epochs=100,           # Early Stopping이 있으므로 에포크를 넉넉히 잡음
    batch_size=32,
    validation_split=0.2, # 데이터의 20%를 검증용으로 사용
    callbacks=[early_stopping, checkpoint],
    verbose=1
)

Epoch 1/100
44/45 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.6309

45/45 ━━━━━━━━━━━━━━━━━━━━ 6s 63ms/step - loss: 0.6283 - val_loss: 0.5005
Epoch 2/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 0.4809

45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 61ms/step - loss: 0.4808 - val_loss: 0.5003
Epoch 3/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 5s 62ms/step - loss: 0.4776 - val_loss: 0.5019
Epoch 4/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 56ms/step - loss: 0.4710 - val_loss: 0.5114
Epoch 5/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 54ms/step - loss: 0.4754 - val_loss: 0.5049
Epoch 6/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 67ms/step - loss: 0.4751 - val_loss: 0.5045
Epoch 7/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 58ms/step - loss: 0.4788 - val_loss: 0.5042
Epoch 8/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 5s 57ms/step - loss: 0.4703 - val_loss: 0.5023
Epoch 9/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 68ms/step - loss: 0.4714 - val_loss: 0.5066
Epoch 10/100
44/45 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 0.4747

45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 58ms/step - loss: 0.4747 - val_loss: 0.4981
Epoch 11/100
44/45 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 0.4764

45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 56ms/step - loss: 0.4763 - val_loss: 0.4976
Epoch 12/100
44/45 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 0.4783

...

45/45 ━━━━━━━━━━━━━━━━━━━━ 5s 54ms/step - loss: 0.4681 - val_loss: 0.4942
Epoch 18/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 55ms/step - loss: 0.4719 - val_loss: 0.4974
Epoch 19/100
44/45 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 0.4751

45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 57ms/step - loss: 0.4750 - val_loss: 0.4933
Epoch 20/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 66ms/step - loss: 0.4770 - val_loss: 0.4934
Epoch 21/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 53ms/step - loss: 0.4711 - val_loss: 0.4949
Epoch 22/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 56ms/step - loss: 0.4691 - val_loss: 0.4986
Epoch 23/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 56ms/step - loss: 0.4730 - val_loss: 0.4940
Epoch 24/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 0.4631

45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 64ms/step - loss: 0.4634 - val_loss: 0.4931
Epoch 25/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 64ms/step - loss: 0.4682 - val_loss: 0.4950
Epoch 26/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 60ms/step - loss: 0.4733 - val_loss: 0.4936
Epoch 27/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 59ms/step - loss: 0.4675 - val_loss: 0.4944
Epoch 28/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 57ms/step - loss: 0.4738 - val_loss: 0.4943
Epoch 29/100
44/45 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - loss: 0.4765

45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 68ms/step - loss: 0.4763 - val_loss: 0.4924
Epoch 30/100
44/45 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - loss: 0.4797

45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 58ms/step - loss: 0.4794 - val_loss: 0.4923
Epoch 31/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 5s 54ms/step - loss: 0.4746 - val_loss: 0.4928
Epoch 32/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 64ms/step - loss: 0.4785 - val_loss: 0.4930
Epoch 33/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 56ms/step - loss: 0.4803 - val_loss: 0.4926
Epoch 34/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 54ms/step - loss: 0.4736 - val_loss: 0.4956
Epoch 35/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 55ms/step - loss: 0.4822 - val_loss: 0.4954
Epoch 36/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 59ms/step - loss: 0.4667 - val_loss: 0.4932
Epoch 37/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 5s 53ms/step - loss: 0.4697 - val_loss: 0.4927
Epoch 38/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 54ms/step - loss: 0.4730 - val_loss: 0.4928
Epoch 39/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 63ms/step - loss: 0.4763 - val_loss: 0.4944
Epoch 40/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 66ms/step - loss: 0.4817 - val_loss: 0.4962


In [38]:
import numpy as np

# 1. 초기 설정
balance = 10000000
position = 0
history = []

# 2. 백테스팅 루프 (학습 데이터 이후 시점부터 시작)
for i in range(prediction_days, len(df)):
    current_price = df['Close'].iloc[i]

    # AI 예측: 확률값 가져오기 (0~1 사이)
    last_60_days = scaled_data[i-60:i, 0]
    last_60_days_input = np.reshape(last_60_days, (1, 60, 1))
    prob = model.predict(last_60_days_input, verbose=0)[0][0] # Sigmoid 결과값

    # 지표 데이터
    curr_ma5 = df['MA5'].iloc[i]
    curr_ma20 = df['MA20'].iloc[i]
    prev_ma20 = df['MA20'].iloc[i-1] # 20일선 기울기 확인용

    # [과제 기반 3중 필터 + AI 확률]
    # 1) 가격이 20일선 위 2) 20일선이 우상향 3) 5일선 > 20일선
    is_trend_up = (current_price > curr_ma20) and (curr_ma20 > prev_ma20)
    is_golden_status = (curr_ma5 > curr_ma20)

    # AI 조건: 확률이 0.465를 넘는지 확인
    is_ai_approve = prob > 0.38

    if is_golden_status:
        print(f"[{df.index[i].date()}] 골든 상태! | AI확률: {prob:.4f} | 이평기울기: {'상승' if curr_ma20 > prev_ma20 else '하락'}")

    # 최종 매수 결정
    if position == 0 and is_trend_up and is_golden_status and is_ai_approve:
        buy_price = current_price
        position = balance / buy_price
        balance = 0
        print(f"🚀 [{df.index[i].date()}] 진입! | 가격: {buy_price:,.0f} | 확률: {prob:.4f}")

    # [매도 조건: 과제 로직 - 익절 15%, 손절 -2%, 추세이탈 ma20]
    elif position > 0:
        profit_loss = (current_price - buy_price) / buy_price

        # A. 익절 15% / B. 손절 -2% / C. 가격이 20일선 하향 돌파
        if profit_loss >= 0.15 or profit_loss <= -0.02 or current_price < curr_ma20:
            balance = position * current_price
            history.append(profit_loss)
            reason = "익절" if profit_loss >= 0.15 else "손절" if profit_loss <= -0.02 else "추세이탈"
            print(f"💰 [{df.index[i].date()}] 매도 ({reason}) | 수익률: {profit_loss*100:.2f}%")
            position = 0

# 최종 결과 출력
print(f"\n최종 수익률: {(balance - 10000000)/100000:.2f}%")

[2025-04-14] 골든 상태! | AI확률: 0.4230 | 이평기울기: 하락
[2025-04-15] 골든 상태! | AI확률: 0.4349 | 이평기울기: 하락
[2025-04-16] 골든 상태! | AI확률: 0.4423 | 이평기울기: 하락
[2025-04-17] 골든 상태! | AI확률: 0.4400 | 이평기울기: 상승
🚀 [2025-04-17] 진입! | 가격: 84,896 | 확률: 0.4400
[2025-04-18] 골든 상태! | AI확률: 0.4313 | 이평기울기: 상승
[2025-04-19] 골든 상태! | AI확률: 0.4091 | 이평기울기: 상승
[2025-04-20] 골든 상태! | AI확률: 0.3859 | 이평기울기: 상승
[2025-04-21] 골든 상태! | AI확률: 0.3725 | 이평기울기: 상승
[2025-04-22] 골든 상태! | AI확률: 0.3601 | 이평기울기: 상승
[2025-04-23] 골든 상태! | AI확률: 0.3448 | 이평기울기: 상승
[2025-04-24] 골든 상태! | AI확률: 0.3310 | 이평기울기: 상승
[2025-04-25] 골든 상태! | AI확률: 0.3195 | 이평기울기: 상승
[2025-04-26] 골든 상태! | AI확률: 0.3096 | 이평기울기: 상승
[2025-04-27] 골든 상태! | AI확률: 0.3190 | 이평기울기: 상승
[2025-04-28] 골든 상태! | AI확률: 0.3384 | 이평기울기: 상승
[2025-04-29] 골든 상태! | AI확률: 0.3540 | 이평기울기: 상승
[2025-04-30] 골든 상태! | AI확률: 0.3579 | 이평기울기: 상승
[2025-05-01] 골든 상태! | AI확률: 0.3690 | 이평기울기: 상승
[2025-05-02] 골든 상태! | AI확률: 0.3811 | 이평기울기: 상승
[2025-05-03] 골든 상태! | AI확률: 0.3851 | 이평기울기: 상승
[2025-05-04] 골든

In [39]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

pio.renderers.default = 'colab'

# 1. 시뮬레이션 및 데이터 기록용 변수
initial_balance = 10000000
balance_v4 = initial_balance
position_v4 = 0
buy_price = 0

# 그래프용 기록 리스트
dates = []
limitless_values = []
bh_values = []
buy_markers_x = []
buy_markers_y = []
sell_markers_x = []
sell_markers_y = []

# Buy & Hold 초기 설정
first_idx = prediction_days
bh_initial_price = df['Close'].iloc[first_idx]
bh_position = initial_balance / bh_initial_price

# 2. 백테스팅 및 일별 자산 기록 루프
for i in range(first_idx, len(df)):
    current_date = df.index[i]
    current_price = df['Close'].iloc[i]

    # AI 및 지표 데이터 (이전 성공 로직 그대로 사용)
    last_60_days = scaled_data[i-60:i, 0]
    last_60_days_input = np.reshape(last_60_days, (1, 60, 1))
    prob = model.predict(last_60_days_input, verbose=0)[0][0]

    curr_ma5 = df['MA5'].iloc[i]
    curr_ma20 = df['MA20'].iloc[i]
    prev_ma20 = df['MA20'].iloc[i-1]

    # [매수/매도 로직]
    is_trend_up = (current_price > curr_ma20) and (curr_ma20 > prev_ma20)
    is_golden_status = (curr_ma5 > curr_ma20)
    is_ai_approve = prob > 0.38 # 24%를 만든 그 기준값!

    if position_v4 == 0 and is_trend_up and is_golden_status and is_ai_approve:
        buy_price = current_price
        position_v4 = balance_v4 / buy_price
        balance_v4 = 0
        buy_markers_x.append(current_date)
        buy_markers_y.append(current_price)

    elif position_v4 > 0:
        profit_loss = (current_price - buy_price) / buy_price
        if profit_loss >= 0.15 or profit_loss <= -0.02 or current_price < curr_ma20:
            balance_v4 = position_v4 * current_price
            position_v4 = 0
            sell_markers_x.append(current_date)
            sell_markers_y.append(current_price)

    # 매일매일의 가치 기록
    dates.append(current_date)
    limitless_values.append(balance_v4 + (position_v4 * current_price))
    bh_values.append(bh_position * current_price)

# 3. 통합 차트 생성
fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                    vertical_spacing=0.05,
                    subplot_titles=('Limitless: BTC 시세 및 매매 타점', '누적 수익률 비교 (%)'),
                    row_heights=[0.6, 0.4])

# (1) 상단: 캔들스틱 + 이평선 + 매매마커
fig.add_trace(go.Candlestick(x=df.index[first_idx:], open=df['Open'].iloc[first_idx:], high=df['High'].iloc[first_idx:], low=df['Low'].iloc[first_idx:], close=df['Close'].iloc[first_idx:], name="BTC", opacity=0.4), row=1, col=1)
fig.add_trace(go.Scatter(x=df.index[first_idx:], y=df['MA20'].iloc[first_idx:], name='MA20', line=dict(color='cyan', width=1)), row=1, col=1)

# 매수/매도 마커 추가
fig.add_trace(go.Scatter(x=buy_markers_x, y=buy_markers_y, mode='markers', marker=dict(symbol='triangle-up', size=12, color='lime'), name='매수(Entry)'), row=1, col=1)
fig.add_trace(go.Scatter(x=sell_markers_x, y=sell_markers_y, mode='markers', marker=dict(symbol='triangle-down', size=12, color='red'), name='매도(Exit)'), row=1, col=1)

# (2) 하단: 수익률 곡선 (%)
limitless_returns = [(v/initial_balance - 1)*100 for v in limitless_values]
bh_returns = [(v/initial_balance - 1)*100 for v in bh_values]

fig.add_trace(go.Scatter(x=dates, y=limitless_returns, name='Limitless 전략', line=dict(color='lime', width=2)), row=2, col=1)
fig.add_trace(go.Scatter(x=dates, y=bh_returns, name='Buy & Hold', line=dict(color='gray', width=1, dash='dash')), row=2, col=1)

fig.update_layout(height=800, title='<b>Limitless v4.2</b> Performance Dashboard', template='plotly_dark', xaxis_rangeslider_visible=False)
fig.show()

## 다중 피처 학습(Multi-Feature)

In [40]:
# 1. 추가 데이터 계산
df['RSI'] = ta.rsi(df['Close'], length=14)
df['Vol_Change'] = df['Volume'].pct_change() # 거래량 변화율 추가

# 결측치 제거 및 필요한 컬럼만 추출
features = df[['Close', 'RSI', 'Volume']].dropna()

# 2. 데이터 정규화 (각 피처별로 따로 스케일링)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_features = scaler.fit_transform(features)

# 3. 학습 데이터셋 생성 (입력 차원이 1에서 3으로 변경됨)
prediction_days = 60
x_train, y_train = [], []

for x in range(prediction_days, len(scaled_features)):
    # x-60일부터 x일까지의 [Close, RSI, Volume] 3가지 데이터를 입력으로 사용
    x_train.append(scaled_features[x-prediction_days:x, :])
    # 정답은 여전히 'Close' 가격의 다음 날 값 (첫 번째 컬럼)
    y_train.append(scaled_features[x, 0])

x_train, y_train = np.array(x_train), np.array(y_train)

In [41]:
model = Sequential()

# input_shape=(60, 3) -> 60일치 데이터, 3개의 피처
model.add(GRU(units=50, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(Dropout(0.2))
model.add(GRU(units=50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=1, activation='sigmoid')) # 과제 로직인 확률 출력을 위해 sigmoid 사용

model.compile(optimizer='adam', loss='binary_crossentropy') # 분류 모델에 최적화된 손실함수

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



In [42]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)


# 3. 모델 학습 실행
history = model.fit(
    x_train, y_train,
    epochs=100,
    batch_size=32,
    validation_split=0.2, # 데이터의 20%를 검증용으로 사용
    callbacks=[early_stopping, checkpoint],
    verbose=1
)

Epoch 1/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 110ms/step - loss: 0.6768 - val_loss: 0.8215
Epoch 2/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step - loss: 0.6491 - val_loss: 0.7869
Epoch 3/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 0.6300 - val_loss: 0.7071
Epoch 4/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - loss: 0.6007 - val_loss: 0.6381
Epoch 5/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 0.5804 - val_loss: 0.5932
Epoch 6/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - loss: 0.5839 - val_loss: 0.5955
Epoch 7/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - loss: 0.5687 - val_loss: 0.6008
Epoch 8/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - loss: 0.5710 - val_loss: 0.5953
Epoch 9/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 0.5689 - val_loss: 0.5942
Epoch 10/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - loss: 0.5684 - val_loss: 0.5936
Epoch 11/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 90ms/step - loss: 0.5610 - val_loss: 0.5928
Epoch 12/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 93ms/step - loss: 0.5773 - val_l

In [43]:
import numpy as np

# 1. 초기 설정
balance = 10000000
position = 0
history = []

# 2. 백테스팅 루프 (학습 데이터 이후 시점부터 시작)
for i in range(prediction_days, len(df)):
    current_price = df['Close'].iloc[i]

    # AI 예측: 확률값 가져오기 (0~1 사이)
    last_60_days = scaled_features[i-60:i, :]
    last_60_days_input = np.reshape(last_60_days, (1, 60, 3))
    prob = model.predict(last_60_days_input, verbose=0)[0][0] # Sigmoid 결과값

    # 지표 데이터
    curr_ma5 = df['MA5'].iloc[i]
    curr_ma20 = df['MA20'].iloc[i]
    prev_ma20 = df['MA20'].iloc[i-1] # 20일선 기울기 확인용

    # [과제 기반 3중 필터 + AI 확률]
    # 1) 가격이 20일선 위 2) 20일선이 우상향 3) 5일선 > 20일선
    is_trend_up = (current_price > curr_ma20) and (curr_ma20 > prev_ma20)
    is_golden_status = (curr_ma5 > curr_ma20)

    # AI 조건: 확률이 0.465를 넘는지 확인
    is_ai_approve = prob > 0.38

    if is_golden_status:
        print(f"[{df.index[i].date()}] 골든 상태! | AI확률: {prob:.4f} | 이평기울기: {'상승' if curr_ma20 > prev_ma20 else '하락'}")

    # 최종 매수 결정
    if position == 0 and is_trend_up and is_golden_status and is_ai_approve:
        buy_price = current_price
        position = balance / buy_price
        balance = 0
        print(f"🚀 [{df.index[i].date()}] 진입! | 가격: {buy_price:,.0f} | 확률: {prob:.4f}")

    # [매도 조건: 과제 로직 - 익절 15%, 손절 -2%, 추세이탈 ma20]
    elif position > 0:
        profit_loss = (current_price - buy_price) / buy_price

        # A. 익절 15% / B. 손절 -2% / C. 가격이 20일선 하향 돌파
        if profit_loss >= 0.15 or profit_loss <= -0.02 or current_price < curr_ma20:
            balance = position * current_price
            history.append(profit_loss)
            reason = "익절" if profit_loss >= 0.15 else "손절" if profit_loss <= -0.02 else "추세이탈"
            print(f"💰 [{df.index[i].date()}] 매도 ({reason}) | 수익률: {profit_loss*100:.2f}%")
            position = 0

# 최종 결과 출력
print(f"\n최종 수익률: {(balance - 10000000)/100000:.2f}%")

[2025-04-14] 골든 상태! | AI확률: 0.1909 | 이평기울기: 하락
[2025-04-15] 골든 상태! | AI확률: 0.1883 | 이평기울기: 하락
[2025-04-16] 골든 상태! | AI확률: 0.1885 | 이평기울기: 하락
[2025-04-17] 골든 상태! | AI확률: 0.1982 | 이평기울기: 상승
[2025-04-18] 골든 상태! | AI확률: 0.2001 | 이평기울기: 상승
[2025-04-19] 골든 상태! | AI확률: 0.2040 | 이평기울기: 상승
[2025-04-20] 골든 상태! | AI확률: 0.2079 | 이평기울기: 상승
[2025-04-21] 골든 상태! | AI확률: 0.2266 | 이평기울기: 상승
[2025-04-22] 골든 상태! | AI확률: 0.3028 | 이평기울기: 상승
[2025-04-23] 골든 상태! | AI확률: 0.3550 | 이평기울기: 상승
[2025-04-24] 골든 상태! | AI확률: 0.3723 | 이평기울기: 상승
[2025-04-25] 골든 상태! | AI확률: 0.3787 | 이평기울기: 상승
[2025-04-26] 골든 상태! | AI확률: 0.3926 | 이평기울기: 상승
🚀 [2025-04-26] 진입! | 가격: 94,647 | 확률: 0.3926
[2025-04-27] 골든 상태! | AI확률: 0.3723 | 이평기울기: 상승
[2025-04-28] 골든 상태! | AI확률: 0.3775 | 이평기울기: 상승
[2025-04-29] 골든 상태! | AI확률: 0.3777 | 이평기울기: 상승
[2025-04-30] 골든 상태! | AI확률: 0.3742 | 이평기울기: 상승
[2025-05-01] 골든 상태! | AI확률: 0.4073 | 이평기울기: 상승
[2025-05-02] 골든 상태! | AI확률: 0.4340 | 이평기울기: 상승
[2025-05-03] 골든 상태! | AI확률: 0.4217 | 이평기울기: 상승
[2025-05-04] 골든

In [44]:
pio.renderers.default = 'colab'

# 1. 시뮬레이션 및 데이터 기록용 변수
initial_balance = 10000000
balance_v4 = initial_balance
position_v4 = 0
buy_price = 0

# 그래프용 기록 리스트
dates = []
limitless_values = []
bh_values = []
buy_markers_x = []
buy_markers_y = []
sell_markers_x = []
sell_markers_y = []

# Buy & Hold 초기 설정
first_idx = prediction_days
bh_initial_price = df['Close'].iloc[first_idx]
bh_position = initial_balance / bh_initial_price

# 2. 백테스팅 및 일별 자산 기록 루프
for i in range(first_idx, len(df)):
    current_date = df.index[i]
    current_price = df['Close'].iloc[i]

    # AI 및 지표 데이터 (이전 성공 로직 그대로 사용)
    last_60_days = scaled_features[i-60:i, :]
    last_60_days_input = np.reshape(last_60_days, (1, 60, 3))
    prob = model.predict(last_60_days_input, verbose=0)[0][0]

    curr_ma5 = df['MA5'].iloc[i]
    curr_ma20 = df['MA20'].iloc[i]
    prev_ma20 = df['MA20'].iloc[i-1]

    # [매수/매도 로직]
    is_trend_up = (current_price > curr_ma20) and (curr_ma20 > prev_ma20)
    is_golden_status = (curr_ma5 > curr_ma20)
    is_ai_approve = prob > 0.38 # 24%를 만든 그 기준값!

    if position_v4 == 0 and is_trend_up and is_golden_status and is_ai_approve:
        buy_price = current_price
        position_v4 = balance_v4 / buy_price
        balance_v4 = 0
        buy_markers_x.append(current_date)
        buy_markers_y.append(current_price)

    elif position_v4 > 0:
        profit_loss = (current_price - buy_price) / buy_price
        if profit_loss >= 0.15 or profit_loss <= -0.02 or current_price < curr_ma20:
            balance_v4 = position_v4 * current_price
            position_v4 = 0
            sell_markers_x.append(current_date)
            sell_markers_y.append(current_price)

    # 매일매일의 가치 기록
    dates.append(current_date)
    limitless_values.append(balance_v4 + (position_v4 * current_price))
    bh_values.append(bh_position * current_price)

# 3. 통합 차트 생성
fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                    vertical_spacing=0.05,
                    subplot_titles=('Limitless: BTC 시세 및 매매 타점', '누적 수익률 비교 (%)'),
                    row_heights=[0.6, 0.4])

# (1) 상단: 캔들스틱 + 이평선 + 매매마커
fig.add_trace(go.Candlestick(x=df.index[first_idx:], open=df['Open'].iloc[first_idx:], high=df['High'].iloc[first_idx:], low=df['Low'].iloc[first_idx:], close=df['Close'].iloc[first_idx:], name="BTC", opacity=0.4), row=1, col=1)
fig.add_trace(go.Scatter(x=df.index[first_idx:], y=df['MA20'].iloc[first_idx:], name='MA20', line=dict(color='cyan', width=1)), row=1, col=1)

# 매수/매도 마커 추가
fig.add_trace(go.Scatter(x=buy_markers_x, y=buy_markers_y, mode='markers', marker=dict(symbol='triangle-up', size=12, color='lime'), name='매수(Entry)'), row=1, col=1)
fig.add_trace(go.Scatter(x=sell_markers_x, y=sell_markers_y, mode='markers', marker=dict(symbol='triangle-down', size=12, color='red'), name='매도(Exit)'), row=1, col=1)

# (2) 하단: 수익률 곡선 (%)
limitless_returns = [(v/initial_balance - 1)*100 for v in limitless_values]
bh_returns = [(v/initial_balance - 1)*100 for v in bh_values]

fig.add_trace(go.Scatter(x=dates, y=limitless_returns, name='Limitless 전략', line=dict(color='lime', width=2)), row=2, col=1)
fig.add_trace(go.Scatter(x=dates, y=bh_returns, name='Buy & Hold', line=dict(color='gray', width=1, dash='dash')), row=2, col=1)

fig.update_layout(height=800, title='<b>Limitless v4.2</b> Performance Dashboard', template='plotly_dark', xaxis_rangeslider_visible=False)
fig.show()

## 분할 매수 기법 적용 (캘리 공식)

In [45]:
# 1. 초기 설정
balance = 10000000
position_amount = 0  # 현재 보유 중인 코인의 개수
history = []
win_rate = 0.45       # 예상 승률 (이전 백테스팅 기반)
profit_loss_ratio = 3.0 # 평균 손익비 (익절 15% / 손절 5% 가정)

# 2. 백테스팅 루프
for i in range(prediction_days, len(df)):
    current_price = df['Close'].iloc[i]

    # AI 예측 데이터 준비 (3개 피처: Price, RSI, Volume)
    last_60_days = scaled_features[i-60:i, :]
    last_60_days_input = np.reshape(last_60_days, (1, 60, 3))
    prob = model.predict(last_60_days_input, verbose=0)[0][0]

    # 기본 필터
    curr_ma20 = df['MA20'].iloc[i]
    prev_ma20 = df['MA20'].iloc[i-1]
    is_trend_up = (current_price > curr_ma20) and (curr_ma20 > prev_ma20)

    # [켈리 공식 적용]
    # 투자 비중 계산: f = (prob * (b + 1) - 1) / b
    kelly_f = (prob * (profit_loss_ratio + 1) - 1) / profit_loss_ratio

    # 안정성을 위해 계산된 비중의 30%만 사용 (프랙셔널 켈리)
    invest_ratio = max(0, kelly_f * 0.3)
    if invest_ratio > 1.0: invest_ratio = 1.0

    # 매수 로직 (분할 진입)
    if is_trend_up and prob > 0.4 and balance > 100000: # 최소 10만원 이상 잔고 있을 때
        amount_to_invest = balance * invest_ratio
        new_position = amount_to_invest / current_price

        position_amount += new_position
        balance -= amount_to_invest

        # 진입 시점 기록 (첫 진입 시에만 buy_price 설정)
        if 'buy_price' not in locals() or position_amount == new_position:
            buy_price = current_price

        print(f"➕ [{df.index[i].date()}] 분할 매수 | 비중: {invest_ratio*100:.1f}% | 잔고: {balance:,.0f}")

    # 매도 로직 (전량 매도 - 리스크 관리)
    elif position_amount > 0:
        current_return = (current_price - buy_price) / buy_price

        # 익절/손절/추세이탈 시 전량 탈출
        if current_return >= 0.15 or current_return <= -0.03 or current_price < curr_ma20:
            balance += position_amount * current_price
            reason = "익절" if current_return >= 0.15 else "손절" if current_return <= -0.03 else "추세이탈"
            print(f"💰 [{df.index[i].date()}] 전량 매도({reason}) | 수익률: {current_return*100:.2f}% | 최종잔고: {balance:,.0f}")

            position_amount = 0
            del buy_price # 매도 후 매수가 초기화

➕ [2025-05-01] 분할 매수 | 비중: 6.3% | 잔고: 9,370,989
➕ [2025-05-02] 분할 매수 | 비중: 7.4% | 잔고: 8,681,105
➕ [2025-05-03] 분할 매수 | 비중: 6.9% | 잔고: 8,084,916
➕ [2025-05-06] 분할 매수 | 비중: 6.4% | 잔고: 7,566,866
➕ [2025-05-07] 분할 매수 | 비중: 6.5% | 잔고: 7,071,982
➕ [2025-05-08] 분할 매수 | 비중: 11.4% | 잔고: 6,264,704
➕ [2025-05-09] 분할 매수 | 비중: 13.2% | 잔고: 5,436,918
➕ [2025-05-10] 분할 매수 | 비중: 14.5% | 잔고: 4,647,973
➕ [2025-05-11] 분할 매수 | 비중: 14.0% | 잔고: 3,997,607
➕ [2025-05-12] 분할 매수 | 비중: 12.4% | 잔고: 3,502,214
➕ [2025-05-13] 분할 매수 | 비중: 13.4% | 잔고: 3,032,176
➕ [2025-05-14] 분할 매수 | 비중: 13.3% | 잔고: 2,629,582
➕ [2025-05-15] 분할 매수 | 비중: 13.1% | 잔고: 2,284,098
➕ [2025-05-16] 분할 매수 | 비중: 13.1% | 잔고: 1,983,873
➕ [2025-05-17] 분할 매수 | 비중: 12.9% | 잔고: 1,727,244
➕ [2025-05-18] 분할 매수 | 비중: 14.8% | 잔고: 1,471,733
➕ [2025-05-19] 분할 매수 | 비중: 14.9% | 잔고: 1,252,962
➕ [2025-05-20] 분할 매수 | 비중: 16.3% | 잔고: 1,048,716
➕ [2025-05-21] 분할 매수 | 비중: 17.4% | 잔고: 865,835
➕ [2025-05-22] 분할 매수 | 비중: 19.8% | 잔고: 694,274
➕ [2025-05-23] 분할 매수 | 비중: 17

잦은 추세이탈 매도로 인해 수익이 나지 않는 것이라고 판단되었음

   
분할 매도 도입 시도

In [46]:
# 1. 초기 설정 및 변수 추가
balance = 10000000
position_amount = 0
half_sold = False  # 분할 매도 여부 체크용 플래그
history = []

# 2. 백테스팅 루프
for i in range(prediction_days, len(df)):
    current_price = df['Close'].iloc[i]

    # AI 예측 및 지표 데이터 (이전과 동일)
    last_60_days = scaled_features[i-60:i, :]
    last_60_days_input = np.reshape(last_60_days, (1, 60, 3))
    prob = model.predict(last_60_days_input, verbose=0)[0][0]

    curr_ma20 = df['MA20'].iloc[i]
    prev_ma20 = df['MA20'].iloc[i-1]
    is_trend_up = (current_price > curr_ma20) and (curr_ma20 > prev_ma20)

    # [매수 로직: 켈리 공식 기반 분할 매수]
    if is_trend_up and prob > 0.4 and balance > 100000:
        kelly_f = (prob * 4 - 1) / 3 # 손익비 조절
        invest_ratio = max(0, kelly_f * 0.3)

        amount_to_invest = balance * invest_ratio
        new_position = amount_to_invest / current_price

        if position_amount == 0:
            buy_price = current_price
            half_sold = False # 새 포지션 진입 시 플래그 초기화

        position_amount += new_position
        balance -= amount_to_invest
        print(f"➕ [{df.index[i].date()}] 분할 매수 | 비중: {invest_ratio*100:.1f}%")

    # [매도 로직: 분할 매도 + 전량 매도]
    elif position_amount > 0:
        current_return = (current_price - buy_price) / buy_price

        # A. [분할 매도] 수익률 7% 돌파 시 50% 익절
        if current_return >= 0.07 and not half_sold:
            sell_amount = position_amount * 0.5
            balance += sell_amount * current_price
            position_amount -= sell_amount
            half_sold = True
            print(f"✂️ [{df.index[i].date()}] 1차 분할 익절(50%) | 수익률: {current_return*100:.2f}%")

        # B. [전량 매도] 최종 익절(15%), 손절(-3%), 또는 추세 이탈
        elif current_return >= 0.15 or current_return <= -0.03 or current_price < curr_ma20:
            balance += position_amount * current_price
            reason = "최종익절" if current_return >= 0.15 else "손절" if current_return <= -0.03 else "추세이탈"
            print(f"💰 [{df.index[i].date()}] 전량 매도({reason}) | 수익률: {current_return*100:.2f}% | 잔고: {balance:,.0f}")
            position_amount = 0

➕ [2025-05-01] 분할 매수 | 비중: 6.3%
➕ [2025-05-02] 분할 매수 | 비중: 7.4%
➕ [2025-05-03] 분할 매수 | 비중: 6.9%
➕ [2025-05-06] 분할 매수 | 비중: 6.4%
➕ [2025-05-07] 분할 매수 | 비중: 6.5%
➕ [2025-05-08] 분할 매수 | 비중: 11.4%
➕ [2025-05-09] 분할 매수 | 비중: 13.2%
➕ [2025-05-10] 분할 매수 | 비중: 14.5%
➕ [2025-05-11] 분할 매수 | 비중: 14.0%
➕ [2025-05-12] 분할 매수 | 비중: 12.4%
➕ [2025-05-13] 분할 매수 | 비중: 13.4%
➕ [2025-05-14] 분할 매수 | 비중: 13.3%
➕ [2025-05-15] 분할 매수 | 비중: 13.1%
➕ [2025-05-16] 분할 매수 | 비중: 13.1%
➕ [2025-05-17] 분할 매수 | 비중: 12.9%
➕ [2025-05-18] 분할 매수 | 비중: 14.8%
➕ [2025-05-19] 분할 매수 | 비중: 14.9%
➕ [2025-05-20] 분할 매수 | 비중: 16.3%
➕ [2025-05-21] 분할 매수 | 비중: 17.4%
➕ [2025-05-22] 분할 매수 | 비중: 19.8%
➕ [2025-05-23] 분할 매수 | 비중: 17.0%
➕ [2025-05-24] 분할 매수 | 비중: 16.5%
➕ [2025-05-25] 분할 매수 | 비중: 17.0%
➕ [2025-05-26] 분할 매수 | 비중: 17.7%
➕ [2025-05-27] 분할 매수 | 비중: 17.5%
➕ [2025-05-28] 분할 매수 | 비중: 16.9%
✂️ [2025-05-29] 1차 분할 익절(50%) | 수익률: 9.48%
💰 [2025-05-30] 전량 매도(추세이탈) | 수익률: 7.78% | 잔고: 10,267,549
➕ [2025-06-09] 분할 매수 | 비중: 17.6%
➕ [2025-06-10]

In [47]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# 1. 기록용 변수 초기화 (매우 중요)
initial_balance = 10000000
balance = initial_balance
position_amount = 0
half_sold = False
buy_price = 0

dates_log = []
portfolio_returns = []
bh_returns = []

# 매매 마커 기록용
buy_x, buy_y = [], []
partial_x, partial_y = [], []
exit_x, exit_y = [], []

# Buy & Hold 기준가
bh_start_price = df['Close'].iloc[prediction_days]

# 2. 백테스팅 루프 (v4.5 로직 통합)
for i in range(prediction_days, len(df)):
    curr_date = df.index[i]
    curr_price = df['Close'].iloc[i]

    # AI 예측 (3개 피처 입력)
    last_60 = scaled_features[i-60:i, :]
    prob = model.predict(last_60.reshape(1, 60, 3), verbose=0)[0][0]

    # 지표
    ma20 = df['MA20'].iloc[i]
    prev_ma20 = df['MA20'].iloc[i-1]
    is_trend_up = (curr_price > ma20) and (ma20 > prev_ma20)

    # [매수 로직]
    if is_trend_up and prob > 0.4 and balance > 100000:
        kelly_f = (prob * 4 - 1) / 3
        invest_ratio = max(0, kelly_f * 0.3)
        invest_amt = balance * invest_ratio

        if position_amount == 0:
            buy_price = curr_price
            half_sold = False

        position_amount += invest_amt / curr_price
        balance -= invest_amt
        buy_x.append(curr_date); buy_y.append(curr_price)

    # [매도 로직]
    elif position_amount > 0:
        ret = (curr_price - buy_price) / buy_price
        # 1차 익절
        if ret >= 0.07 and not half_sold:
            balance += (position_amount * 0.5) * curr_price
            position_amount *= 0.5
            half_sold = True
            partial_x.append(curr_date); partial_y.append(curr_price)
        # 전량 매도
        elif ret >= 0.15 or ret <= -0.03 or curr_price < ma20:
            balance += position_amount * curr_price
            position_amount = 0
            exit_x.append(curr_date); exit_y.append(curr_price)

    # [수익률 기록] - 이 부분이 그래프의 생명입니다.
    current_val = balance + (position_amount * curr_price)
    portfolio_returns.append(((current_val / initial_balance) - 1) * 100)
    bh_returns.append(((curr_price / bh_start_price) - 1) * 100)
    dates_log.append(curr_date)

# 3. 시각화
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05,
                    subplot_titles=('Limitless v4.5 매매 타점', '누적 수익률 비교 (%)'))

# 상단: 가격 + 이평선 + 마커
fig.add_trace(go.Scatter(x=df.index[prediction_days:], y=df['Close'].iloc[prediction_days:], name='BTC Price', line=dict(color='gray', width=1), opacity=0.5), row=1, col=1)
fig.add_trace(go.Scatter(x=buy_x, y=buy_y, mode='markers', name='Buy', marker=dict(symbol='triangle-up', color='lime', size=10)), row=1, col=1)
fig.add_trace(go.Scatter(x=partial_x, y=partial_y, mode='markers', name='Partial Sell', marker=dict(symbol='star', color='orange', size=10)), row=1, col=1)
fig.add_trace(go.Scatter(x=exit_x, y=exit_y, mode='markers', name='Exit', marker=dict(symbol='triangle-down', color='red', size=10)), row=1, col=1)

# 하단: 수익률 곡선 (0%에서 변화가 보여야 함)
fig.add_trace(go.Scatter(x=dates_log, y=portfolio_returns, name='Limitless v4.5', line=dict(color='lime', width=2)), row=2, col=1)
fig.add_trace(go.Scatter(x=dates_log, y=bh_returns, name='Buy & Hold', line=dict(color='white', width=1, dash='dot')), row=2, col=1)

fig.update_layout(height=800, template='plotly_dark', title='Limitless Performance Analysis')
fig.show()

분할 매수 전략을 추가하던 중, 가중 평균 단가를 무시한 채 진행한 경우 더 기존 전략 계산 가격보다 더 낮은 가격에서 손절하는 경우 발생   


---



**"Scale-in(분할 매수) 전략에서 가중 평균 단가(WAP) 관리의 중요성"**

단순히 가격 데이터만 활용하는 것이 아니라, 투입된 자본의 가중치를 반영한 실시간 평단가를 계산해야 정확한 리스크 관리(손절)가 가능함.

평단가 업데이트가 누락될 경우, 모델은 수익권으로 착각하여 과도한 낙폭을 견디게 되며, 이는 전체 자산의 치명적인 손실(Drawdown)로 이어짐.

In [55]:
# 1. 초기 변수 설정
initial_balance = 10000000
balance = initial_balance
position_amount = 0  # 현재 보유 코인 개수
buy_price = 0       # 가중 평균 단가 (평단가)
half_sold = False   # 분할 익절 여부

# 기록용
portfolio_history = []
dates_log = []

for i in range(prediction_days, len(df)):
    curr_date = df.index[i]
    curr_price = df['Close'].iloc[i]

    # AI 및 지표 데이터 (3개 피처 사용)
    last_60 = scaled_features[i-60:i, :]
    prob = model.predict(last_60.reshape(1, 60, 3), verbose=0)[0][0]
    ma20 = df['MA20'].iloc[i]
    prev_ma20 = df['MA20'].iloc[i-1]
    is_trend_up = (curr_price > ma20) and (ma20 > prev_ma20)

    # --- [1단계: 매수 및 평단가 업데이트] ---
    if is_trend_up and prob > 0.45 and balance > 100000:
        kelly_f = (prob * 4 - 1) / 3
        invest_ratio = max(0, kelly_f * 0.3)
        invest_amt = balance * invest_ratio

        new_amount = invest_amt / curr_price # 이번에 새로 사는 코인 개수

        if position_amount == 0:
            # 처음 진입할 때
            buy_price = curr_price
            half_sold = False
        else:
            # [핵심] 추가 매수 시 평단가 갱신: (기존가치 + 신규투자금) / 전체수량
            buy_price = ((position_amount * buy_price) + invest_amt) / (position_amount + new_amount)

        position_amount += new_amount
        balance -= invest_amt
        print(f"➕ [{curr_date.date()}] 분할 매수 | 평단가: {buy_price:,.0f} | 비중: {invest_ratio*100:.1f}%")

    # --- [2단계: 매도 및 리스크 관리] ---
    elif position_amount > 0:
        # 수익률 계산의 기준은 항상 실시간으로 업데이트된 'buy_price'입니다.
        current_return = (curr_price - buy_price) / buy_price

        # A. 분할 익절 로직
        if current_return >= 0.07 and not half_sold:
            balance += (position_amount * 0.5) * curr_price
            position_amount *= 0.5
            half_sold = True
            print(f"✂️ [{curr_date.date()}] 1차 분할 익절(50%) | 수익률: {current_return*100:.2f}%")

        # B. 리스크 관리 로직 (본절가 보호 적용)
        stop_loss_limit = 0.0 if half_sold else -0.03 # 1차 익절 후엔 본절 사수

        if current_return >= 0.15:
            reason = "최종익절"
        elif current_return <= stop_loss_limit:
            reason = "본절방어" if half_sold else "손절"
        elif curr_price < ma20:
            reason = "추세이탈"
        else:
            reason = None

        if reason:
            balance += position_amount * curr_price
            print(f"💰 [{curr_date.date()}] 전량 매도({reason}) | 최종수익률: {current_return*100:.2f}% | 잔고: {balance:,.0f}")
            position_amount = 0
            buy_price = 0 # 평단가 초기화

    # 자산 기록
    total_val = balance + (position_amount * curr_price)
    portfolio_history.append(((total_val / initial_balance) - 1) * 100)
    dates_log.append(curr_date)

➕ [2025-05-08] 분할 매수 | 평단가: 103,241 | 비중: 11.4%
➕ [2025-05-09] 분할 매수 | 평단가: 103,104 | 비중: 13.2%
➕ [2025-05-10] 분할 매수 | 평단가: 103,617 | 비중: 14.5%
➕ [2025-05-11] 분할 매수 | 평단가: 103,720 | 비중: 14.0%
➕ [2025-05-12] 분할 매수 | 평단가: 103,593 | 비중: 12.4%
➕ [2025-05-13] 분할 매수 | 평단가: 103,660 | 비중: 13.4%
➕ [2025-05-14] 분할 매수 | 평단가: 103,649 | 비중: 13.3%
➕ [2025-05-15] 분할 매수 | 평단가: 103,656 | 비중: 13.1%
➕ [2025-05-16] 분할 매수 | 평단가: 103,646 | 비중: 13.1%
➕ [2025-05-17] 분할 매수 | 평단가: 103,624 | 비중: 12.9%
➕ [2025-05-18] 분할 매수 | 평단가: 103,750 | 비중: 14.8%
➕ [2025-05-19] 분할 매수 | 평단가: 103,818 | 비중: 14.9%
➕ [2025-05-20] 분할 매수 | 평단가: 103,916 | 비중: 16.3%
➕ [2025-05-21] 분할 매수 | 평단가: 104,078 | 비중: 17.4%
➕ [2025-05-22] 분할 매수 | 평단가: 104,268 | 비중: 19.8%
➕ [2025-05-23] 분할 매수 | 평단가: 104,322 | 비중: 17.0%
➕ [2025-05-24] 분할 매수 | 평단가: 104,370 | 비중: 16.5%
➕ [2025-05-25] 분할 매수 | 평단가: 104,425 | 비중: 17.0%
➕ [2025-05-26] 분할 매수 | 평단가: 104,475 | 비중: 17.7%
➕ [2025-05-27] 분할 매수 | 평단가: 104,512 | 비중: 17.5%
➕ [2025-05-28] 분할 매수 | 평단가: 104,533 | 비중

보다 안정적인 수익률을 위해 최대 비중 캡 로직을 추가하여 한 번에 보유할 수 있는 최대 비중을 어느 정도 내로 제한하도록 설정함

In [57]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# 1. 초기 설정
initial_balance = 10000000
balance = initial_balance
position_amount = 0
buy_price = 0
half_sold = False

# 리스크 및 수익 설정
MAX_POSITION_RATIO = 0.20  # 안전을 위한 비중 캡 (20%)
TAKE_PROFIT_1 = 0.10       # 1차 익절 라인 상향 (7% -> 10%)

# 기록용
dates_log = []
portfolio_returns = []
bh_returns = []
buy_x, buy_y = [], []
partial_x, partial_y = [], []
exit_x, exit_y = [], []

bh_start_price = df['Close'].iloc[prediction_days]

# 2. 백테스팅 루프
for i in range(prediction_days, len(df)):
    curr_date = df.index[i]
    curr_price = df['Close'].iloc[i]

    # AI 예측 및 지표 (3개 피처)
    last_60 = scaled_features[i-60:i, :]
    prob = model.predict(last_60.reshape(1, 60, 3), verbose=0)[0][0]
    ma20 = df['MA20'].iloc[i]
    prev_ma20 = df['MA20'].iloc[i-1]
    is_trend_up = (curr_price > ma20) and (ma20 > prev_ma20)

    # --- [매수 로직: v4.7의 방어력 유지] ---
    current_val = balance + (position_amount * curr_price)
    current_position_ratio = (position_amount * curr_price) / current_val

    if is_trend_up and prob > 0.38 and balance > 100000: # 문턱을 0.4 -> 0.38로 소폭 완화
        kelly_f = (prob * 4 - 1) / 3
        invest_ratio = max(0, kelly_f * 0.3)

        # Max Cap 20% 적용
        if current_position_ratio >= MAX_POSITION_RATIO:
            invest_ratio = 0
        elif (current_position_ratio + invest_ratio) > MAX_POSITION_RATIO:
            invest_ratio = MAX_POSITION_RATIO - current_position_ratio

        if invest_ratio > 0:
            invest_amt = balance * invest_ratio
            new_amount = invest_amt / curr_price

            # 평단가 업데이트 (WAP)
            if position_amount == 0:
                buy_price = curr_price
                half_sold = False
            else:
                buy_price = ((position_amount * buy_price) + invest_amt) / (position_amount + new_amount)

            position_amount += new_amount
            balance -= invest_amt
            buy_x.append(curr_date); buy_y.append(curr_price)

    # --- [매도 로직: 수익 극대화 모드] ---
    elif position_amount > 0:
        current_return = (curr_price - buy_price) / buy_price

        # 1. 분할 익절 (10% 도달 시 절반 수익 확정)
        if current_return >= TAKE_PROFIT_1 and not half_sold:
            balance += (position_amount * 0.5) * curr_price
            position_amount *= 0.5
            half_sold = True
            partial_x.append(curr_date); partial_y.append(curr_price)
            print(f"✂️ [{curr_date.date()}] 1차 익절 완료! 나머지는 추세 끝까지 홀딩.")

        # 2. 리스크 관리 (본절가 보호 적용)
        stop_loss_limit = 0.0 if half_sold else -0.03

        reason = None
        # [v4.8 변경점] 고정 익절 15%를 삭제하고 오직 손절과 추세이탈만 체크
        if current_return <= stop_loss_limit:
            reason = "본절방어" if half_sold else "손절"
        elif curr_price < ma20:
            reason = "추세이탈(익절)" if current_return > 0 else "추세이탈(손절)"

        if reason:
            balance += position_amount * curr_price
            exit_x.append(curr_date); exit_y.append(curr_price)
            print(f"💰 [{curr_date.date()}] 전량 매도({reason}) | 최종수익률: {current_return*100:.2f}% | 잔고: {balance:,.0f}")
            position_amount = 0
            buy_price = 0

    # 수익률 기록
    portfolio_returns.append(((current_val / initial_balance) - 1) * 100)
    bh_returns.append(((curr_price / bh_start_price) - 1) * 100)
    dates_log.append(curr_date)

# 3. 최종 결과 출력
print(f"\n🚀 [Limitless v4.8] 최종 수익률: {portfolio_returns[-1]:.2f}%")

# 3. 시각화
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05,
                    subplot_titles=('Limitless v4.5 매매 타점', '누적 수익률 비교 (%)'))

# 상단: 가격 + 이평선 + 마커
fig.add_trace(go.Scatter(x=df.index[prediction_days:], y=df['Close'].iloc[prediction_days:], name='BTC Price', line=dict(color='gray', width=1), opacity=0.5), row=1, col=1)
fig.add_trace(go.Scatter(x=buy_x, y=buy_y, mode='markers', name='Buy', marker=dict(symbol='triangle-up', color='lime', size=10)), row=1, col=1)
fig.add_trace(go.Scatter(x=partial_x, y=partial_y, mode='markers', name='Partial Sell', marker=dict(symbol='star', color='orange', size=10)), row=1, col=1)
fig.add_trace(go.Scatter(x=exit_x, y=exit_y, mode='markers', name='Exit', marker=dict(symbol='triangle-down', color='red', size=10)), row=1, col=1)

# 하단: 수익률 곡선 (0%에서 변화가 보여야 함)
fig.add_trace(go.Scatter(x=dates_log, y=portfolio_returns, name='Limitless v4.5', line=dict(color='lime', width=2)), row=2, col=1)
fig.add_trace(go.Scatter(x=dates_log, y=bh_returns, name='Buy & Hold', line=dict(color='white', width=1, dash='dot')), row=2, col=1)

fig.update_layout(height=800, template='plotly_dark', title='Limitless Performance Analysis')
fig.show()

💰 [2025-05-29] 전량 매도(추세이탈(익절)) | 최종수익률: 9.99% | 잔고: 10,198,202
💰 [2025-06-12] 전량 매도(손절) | 최종수익률: -3.95% | 잔고: 10,119,264
✂️ [2025-07-29] 1차 익절 완료! 나머지는 추세 끝까지 홀딩.
💰 [2025-07-29] 전량 매도(추세이탈(익절)) | 최종수익률: 11.47% | 잔고: 10,349,176
💰 [2025-08-18] 전량 매도(추세이탈(손절)) | 최종수익률: -2.56% | 잔고: 10,295,998
💰 [2025-08-23] 전량 매도(추세이탈(손절)) | 최종수익률: -1.28% | 잔고: 10,269,573
💰 [2025-09-22] 전량 매도(추세이탈(손절)) | 최종수익률: -1.06% | 잔고: 10,247,821
💰 [2025-09-30] 전량 매도(추세이탈(손절)) | 최종수익률: -0.30% | 잔고: 10,241,653
💰 [2025-10-10] 전량 매도(손절) | 최종수익률: -4.58% | 잔고: 10,147,832
💰 [2026-01-19] 전량 매도(손절) | 최종수익률: -3.60% | 잔고: 10,075,196

🚀 [Limitless v4.8] 최종 수익률: 0.75%


In [73]:
import yfinance as yf
import pandas as pd
import pandas_ta as ta
import numpy as np
import random
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 1. 랜덤 데이터 로더 함수
def load_random_btc_data(years_back=5, duration_days=365):
    # 최근 n년 내에서 랜덤한 시작일 설정
    end_limit = pd.Timestamp.now() - pd.Timedelta(days=duration_days)
    start_limit = pd.Timestamp.now() - pd.Timedelta(days=years_back * 365)

    random_start = start_limit + (end_limit - start_limit) * random.random()
    random_end = random_start + pd.Timedelta(days=duration_days)

    print(f"📅 테스트 구간: {random_start.date()} ~ {random_end.date()}")

    # 데이터 다운로드 (학습용 60일치 추가로 가져옴)
    data = yf.download("BTC-USD", start=random_start - pd.Timedelta(days=100),
                       end=random_end, interval="1d")
    data.columns = [col[0] if isinstance(col, tuple) else col for col in data.columns]

    # 지표 재계산
    data['MA20'] = ta.sma(data['Close'], length=20)
    data['MA5'] = ta.sma(data['Close'], length=5)
    return data.dropna(), random_start

# 2. 데이터 준비
df, test_start_date = load_random_btc_data(years_back=6, duration_days=365)

# 3. 백테스팅 변수 초기화
initial_balance = 10000000
balance = initial_balance
position_amount = 0
buy_price = 0
half_sold = False

portfolio_returns = []
dates_log = []
buy_x, buy_y, exit_x, exit_y = [], [], [], []

# 4. 백테스팅 루프
for i in range(len(df)):
    curr_date = df.index[i]
    if curr_date < test_start_date: continue # 학습용 버퍼 구간 건너뛰기

    curr_price = df['Close'].iloc[i]
    ma20 = df['MA20'].iloc[i]
    prev_ma20 = df['MA20'].iloc[i-1]

    # [v4.8 로직 적용]
    # 실제 환경에서는 여기서 model.predict가 들어가야 합니다.
    # (과거 데이터 테스트 시에는 미리 학습된 모델을 사용하세요)
    # 임시로 추세 조건만 예시로 넣습니다.
    is_trend_up = (curr_price > ma20) and (ma20 > prev_ma20)

    # 매수 로직 (Max Cap 20%)
    current_val = balance + (position_amount * curr_price)
    if is_trend_up and position_amount == 0 and balance > 100000:
        invest_amt = current_val * 0.20 # 20% 고정 비중 테스트
        position_amount = invest_amt / curr_price
        balance -= invest_amt
        buy_price = curr_price
        half_sold = False
        buy_x.append(curr_date); buy_y.append(curr_price)

    # 매도 로직 (익절 10% 후 추세추종)
    elif position_amount > 0:
        current_return = (curr_price - buy_price) / buy_price

        # 1차 익절
        if current_return >= 0.10 and not half_sold:
            balance += (position_amount * 0.5) * curr_price
            position_amount *= 0.5
            half_sold = True

        # 전량 매도 조건
        stop_limit = 0.0 if half_sold else -0.03
        if current_return <= stop_limit or curr_price < ma20:
            balance += position_amount * curr_price
            exit_x.append(curr_date); exit_y.append(curr_price)
            position_amount = 0
            buy_price = 0

    portfolio_returns.append(((current_val / initial_balance) - 1) * 100)
    dates_log.append(curr_date)

print(f"🏁 최종 누적 수익률: {portfolio_returns[-1]:.2f}%")

/tmp/ipython-input-4227009204.py:21: FutureWarning:

YF.download() has changed argument auto_adjust default to True

/usr/local/lib/python3.12/dist-packages/yfinance/scrapers/history.py:204: Pandas4Warning:

Timestamp.utcnow is deprecated and will be removed in a future version. Use Timestamp.now('UTC') instead.

[*********************100%***********************]  1 of 1 completed

📅 테스트 구간: 2024-02-21 ~ 2025-02-20
🏁 최종 누적 수익률: 5.56%
